In [116]:
import os
import pytesseract
from pdf2image import convert_from_path, convert_from_bytes
import cv2
import re
import pandas as pd
from datetime import datetime
from itertools import zip_longest

class Itagres:
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
    def __init__(self):
       
        self.tesseract_language = "por"
        self.lista_produtos = []
        self.imagem =  r'C:\\Users\\Crawler\\Documents\\DEVPYTHON\brands\\imagens\\'
        self.pdffile =  r'C:\\Users\\Crawler\\Documents\\DEVPYTHON\brands\\ITAGRES.pdf'
        self.dataatual = str(datetime.today().strftime('%Y-%m-%d %H:%M'))
        self.idmarca = '37'

    def converter_imgpdf(self):
        images = convert_from_path(self.pdffile, 200, poppler_path=r'C:\\Users\\Crawler\\Documents\\DEVPYTHON\\brands\\poppler-22.04.0\\Library\\bin')
        for i in range(len(images)):
            images[i].save(self.imagem+r'\itagres'+ str(i) +'.jpg', 'JPEG')

    def search_files(self):
        imagem = [x for x in os.listdir(self.imagem) if 'viscardi']
        return imagem


    #def filtrar_valores(self):
    #    search_valor = re.compile(r'\s+?\d+[A]\s+?')
    #    search_saldo = re.compile(r'\s+?\d+\,\d+\s+?')
    #    alores = search_valor.findall(texto)
    #    saldos = search_saldo.findall(texto)
        

    def reader_imagem(self):
        lista_dicts = []
        imagens = self.search_files()

        #imgs = os.listdir(self.imagem)
        try:
            for im in imagens:
                if 'itagres' in im:
                    img = cv2.imread(os.path.join(self.imagem,im))
                    imagemgray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convertendo para rgb
                    texto = pytesseract.image_to_string(imagemgray
                    , lang= self.tesseract_language,config="tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789,")
                                                    
                    texto = texto.split("\n")
                    remov_blank = list(filter(lambda k: len(k.strip()) >0, texto))
                
                    for values in remov_blank:
                        dict_items = {}
                        cont = len(values)
                        if isinstance(values, str) and cont > 8:
                            values = values.split()
                            try:
                                dict_items['SKU'] = str(values[0].strip())
                            except:
                                dict_items['SKU'] = 'notfound'

                            try:
                                dict_items['SALDO'] = float(str(values[-1].replace("T","").replace(",",".").strip()))
                            except:
                                dict_items['SALDO'] = float(0)

                            dict_items['DATA'] = self.dataatual
                            dict_items['IDMARCA'] = self.idmarca

                            self.lista_produtos.append(dict_items)
                            print(dict_items)
        except:
            print("erro imagem")
                     
    def data_frame_itagres(self):
        try:
            df_itagres = pd.DataFrame(self.lista_produtos)
            df_itagres[df_itagres.loc[:,'SKU'].str.len() >=3]
            df_itagres.groupby(['SKU','SALDO'])[['SKU','SALDO']].sum()
            df_itagres.reset_index()
            
        except:
            print("erro dataframe")
     

        df_itagres.to_excel("itagresdf.xlsx")

 
itagres = Itagres()
itagres.converter_imgpdf()
itagres.reader_imagem()
itagres.data_frame_itagres()
    

{'SKU': 'CÓDIGO', 'SALDO': 0.0, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '17185A', 'SALDO': 57.35, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '04109A', 'SALDO': 4.86, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '04138A', 'SALDO': 61.56, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '07083A', 'SALDO': 1.26, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '04605A', 'SALDO': 145.8, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '04386A', 'SALDO': 134.46, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '04386A', 'SALDO': 24.3, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '04482A', 'SALDO': 3.42, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '17016A', 'SALDO': 11.1, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '41356A', 'SALDO': 28.8, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '41358A', 'SALDO': 64.8, 'DATA': '2022-08-11 18:08', 'IDMARCA': '37'}
{'SKU': '12132A', 'SALDO': 21.84, 'DATA': '2022-08-11 18:08'